# Python Simulation Developement: 


## Event, Segment, Band Naming Convention: 

![BandNamingConventions](Images/EventNaming.jpg)

## High Level Description of Simulation: 

* Each Block is a class that has class methods that (mostly) take a pandas df in, do something to the data and then output a new pandas df. 
* The Simulation class is the main() that calls all of the different blocks. 
    * My hope is that one will be able to read through the Simulation block and get a decent understanding of how the simulation works. 
    * The Simulation Block is also responsible for writing out the different outputs of the blocks to csv's. 
    

## Issues to Address: 

* Need to make it so that the trap is only loaded once. This may entail making all of these blocks inheret from a larger class where all of the configurable parameters and such are loaded as attributes. 
* interp as a parameter to field_profiles. Is this what I want? 
* Why does it not work with 1 event?
* When you impose the power cut in the track builder things start to break. This seems to be a function of the way that the track builder makes tracks from bands. 

## Unit Tests to Create with pytest: 

* Test of the b-field interp consistency. 


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
# Standard:
import os
import json
import math
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

# Supress pd warnings: 
pd.options.mode.chained_assignment = None

# Kris's Library
# import spec_tools.spec_calc.spec_calc as sc
import spec_tools.spec_calc.spec_calc_with_rho_dependence_vectorized as sc
import spec_tools.spec_calc.power_calc as pc
from spec_tools.load_default_field_profiles import load_he6_trap


# Load the Config File : 

In [3]:
# Config File: 
#Loading Simulated Event (JSON)


filepath = "{}/ConfigFiles/SimConfig_Example.json".format(os.getcwd())

with open(filepath,"r") as read_file:
    config_dict = json.load(read_file)
print(config_dict)

{'Configure_Simulation': {'check_for_existing_sim_data': True, 'simulation_results_dir': 'Example_Simulation_dfs', 'simulation_results_file_prefix': 'Not sure I need this'}, 'Physics': {'events_to_simulate': 3, 'monoenergetic': True, 'energy': 17800.0, 'spectrum_isotope': False, 'b_fierz': 0.002, 'weak_magnetism': True, 'min_rho': 0, 'max_rho': 0.00578, 'min_z': -0.005, 'max_z': 0.005, 'min_theta': 85, 'max_theta': 90}, 'Hardware': {'hardware_freqbw_max': 19300000000.0, 'hardware_freqbw_min': 17600000000.0, 'main_field': 0.689, 'trap_strength': 0.001, 'decay_cell_radius': 0.00578, 'base_num_sidebands': 7, 'sideband_tolerance': 0.99, 'standard_decay_cell_radius': True, 'standard_trap_geometry': True, 'calculate_axial_frequencies': True}, 'Kinematics': {'mean_track_length': 0.005, 'jump_size_eV': 0, 'jump_std_eV': 0, 'pitch_angle_costheta_std': 0, 'jump_num_max': 4}, 'BandBuilder': {'sideband_num': 2, 'frac_total_segment_power_cut': 0.0}, 'TrackBuilder': {'trackbuilder_freqbw_max': 19300

## Physics Block: 

In [4]:
class Physics:
    
    """
    An object that generates a beta energy, direction, and initial position, based on the set parameters in config_dict. 
    
    "Physics" : 
        {
        "events_to_simulate" : int; self explanatory.
        "monoenergetic" : bool; if "true", will return "energy". 
        "mono_energy" : float; units (eV)
        } 
    """

    def __init__(self, config_dict):
        
        
        self.Physics_config_dict = config_dict["Physics"]
        self._events_to_simulate = config_dict["Physics"]["events_to_simulate"]
        self._monoenergetic = config_dict["Physics"]["monoenergetic"]
        self._energy = config_dict["Physics"]["energy"]

    def generate_beta_energy(self):
    
        if self.monoenergetic == True: 
            return self.generate_monoenergetic_beta()
        
        else: 
            print("No other options currently configured.")
    
    def generate_monoenergetic_beta(self): 
        return self.energy
    
    def generate_random_spectrum_beta(self): 
        # Fill in later. 
        return 0
    
    def generate_beta_position_direction(self): 
        
        position, direction = sc.random_beta_generator(self.Physics_config_dict) # This previously returned a value of phi = 0 (pos[1]). Ask Kris why. 
        
        return position, direction
    
    @property
    def events_to_simulate(self):
        return self._events_to_simulate  
    
    @property
    def monoenergetic(self):
        return self._monoenergetic
        
    @property
    def energy(self):
        return self._energy

## Hardware Block: 


In [5]:
class Hardware:
    
    """
    An object that takes a beta's position, direction, and energy, along with the parameters from the config_dict and determines if it is trapped. 
    If it is trapped it generates a single row df with all segment properties. 
    
    "Hardware" : {
        "hardware_freqbw_max" : 19.3e9, -> Description here. 
        "hardware_freqbw_min" : 17.6e9, ->
        "main_field" : 0.689, ->
        "trap_strength" : 1e-3, ->
        "base_num_sidebands" : 7, ->
        "sideband_tolerance" : 0.99, ->
        "standard_decay_cell_radius" : true, ->
        "standard_trap_geometry" : true, ->
        "calculate_axial_frequencies" : true ->
        "default_field_profile" : path to field profile? True
    },
    """

    def __init__(self, config_dict):
        
        self.trap_strength = config_dict["Hardware"]["trap_strength"]
        self.main_field = config_dict["Hardware"]["main_field"]
        self.decay_cell_radius = config_dict["Hardware"]["decay_cell_radius"]
        
        # loading trap profile 
        self.trap_profile = load_he6_trap(self.main_field, self.trap_strength)
        self.field_strength = lambda r,z : self.trap_profile.field_values((r,0,z))[2]
        self.trap_profile.initialize_field_strength_interp()


    def construct_untrapped_segment_df(self, beta_position, beta_direction, beta_energy, event_num):
        
        # initial position and direction
        initial_rho_pos = beta_position[0]
        initial_phi_pos = beta_position[1] # position[1] is the phi position. Not physically meaningful right now. Keeping for completion. 
        initial_zpos = beta_position[2] 
        initial_pitch_angle = beta_direction[0]
        initial_phi_dir = beta_direction[1] # Need to be sure this is actually randomized. 
        
        initial_field = self.field_strength(initial_rho_pos, initial_zpos)
        initial_radius = sc.cyc_radius(beta_energy,initial_field,initial_pitch_angle)
        
        center_x = initial_rho_pos - initial_radius * math.cos((90-initial_phi_dir)*math.pi/180)
        center_y = initial_radius * math.sin((90-initial_phi_dir)*math.pi/180)

        rho_center = math.sqrt(center_x**2 + center_y**2)
        
        center_theta = sc.theta_center(initial_zpos, rho_center, initial_pitch_angle,self.trap_profile)
        

        # used for trapping condition (though max radius may be unnecessary)
        trapped_initial_pitch_angle = sc.min_theta(rho_center, initial_zpos,self.trap_profile) # math.asin(math.sqrt(Bz / Bmax)) * 180 / math.pi. Trap profile = 0 implies our "normal" trap. 
        max_radius = sc.max_radius(beta_energy,center_theta,rho_center, self.trap_profile)
        
    
        segment_properties = {"energy": beta_energy,"initial_rho_pos": initial_rho_pos, "initial_phi_pos": initial_phi_pos, "initial_zpos": initial_zpos,"initial_pitch_angle": initial_pitch_angle, 
                              "initial_phi_dir": initial_phi_dir, "center_theta": center_theta, "initial_field":initial_field, "initial_radius": initial_radius,
                              "center_x":center_x, "center_y": center_y, "rho_center": rho_center, "trapped_initial_pitch_angle" : trapped_initial_pitch_angle, 
                              "max_radius": max_radius, "avg_cycl_freq": 0.0, "zmax": 0.0, "axial_freq": 0.0, "mod_index": 0.0, "segment_power": 0.0, 
                              "slope": 0.0, "segment_length": 0.0,"band_power": np.NaN, "band_num": np.NaN,  "segment_num": 0,  "event_num": event_num}
        
        segment_df = pd.DataFrame(segment_properties, index = [event_num])
        
        return segment_df 
    
        
    def trap_condition(self, segment_df):
        
        segment_df = segment_df.reset_index(drop = True)
        
        if segment_df.shape[0] != 1:
            print("ERROR: trap_condition: segment not a single row.  ")
            
        trap_condition = 0 

        if (segment_df["initial_pitch_angle"][0] < segment_df["trapped_initial_pitch_angle"][0]):
            print("Not Trapped: Pitch angle too small.")

            trap_condition += 1 
            
        if (segment_df["rho_center"][0]+segment_df["max_radius"][0] > self.decay_cell_radius):
            print("Not Trapped: Collided with guide wall.")
            trap_condition += 1 

        if trap_condition == 0:
            print("Trapped!")
            return True
        else: 
            return False 


# Kinematics Block 

## Attempt to make this work better: 

Notes or Issues to Address: 

* The trap condition needs to be improved so that it can be used by the scattering/kinematics block. 
    * It works for the most part, it just needs the event_num parameter to be changed so that it works generally for any segment, not just an event. 
    * The issue had to do with matching the index;s
    * AFTER MEETING: Ok going to need to not have the index get reset by the construct untrapped segment function. Just get rid of event_num as a parameter there. 
* Think about having the simulation block be more easily readable... 
* Avoid changing one thing and needing to change a whole bunch of other things. 
* For methods that take a singlerow df as an input, it seems like using df.loc or .iloc is better form. Think about this. 
* Get rid of phi_pos later on now that you've added it in. 
* Is it intuitive that delta_theta be evenly distributed on a sphere? (theta = arccos(2v-1) where v is in [0,1])?
* Vectorize all of the sc funcitons. 
* Difference between rho and rho center? Look into this. 

In [172]:
class Kinematics:
    
    """
    An object containing methods to deal with scattering. 
    
    "Kinematics" : {
        "jump_num" : REDO THIS, 
        "jump_size" : 14,
    },
    """

    def __init__(self, config_dict, hardware):
        
        # Need to address this with inheritance or something: 
        self.hardware = hardware
        
        # Note: Need to get rid of hardware input. Just doing this for now due to the trap profile issue. 
        self.mean_track_length = config_dict["Kinematics"]["mean_track_length"]
        self.jump_size_eV = config_dict["Kinematics"]["jump_size_eV"]
        self.jump_std_eV = config_dict["Kinematics"]["jump_std_eV"]
        self.pitch_angle_costheta_std = config_dict["Kinematics"]["pitch_angle_costheta_std"]
        self.jump_num_max = config_dict["Kinematics"]["jump_num_max"]
        
        # Need this information from "Hardware" as well. 
        self.trap_strength = config_dict["Hardware"]["trap_strength"]
        self.main_field = config_dict["Hardware"]["main_field"]
        self.decay_cell_radius = config_dict["Hardware"]["decay_cell_radius"]
        
        
        # loading trap profile
        # Need to figure out how this will be configurable. For now it is fixed to the default trap profile. 
        # Also need to figure out how this only gets called once for the entire simulation....
        self.trap_profile = hardware.trap_profile
        self.field_strength = lambda r,z : self.trap_profile.field_values((r,0,z))[2]


    def scatter(self, trapped_event_df):
        
        # Constants.
        JOULE_TO_EV = 1/(1.60217733e-19) 
        
        print(trapped_event_df.shape)
        mu = self.mean_track_length
        segment_length = np.random.exponential(mu,trapped_event_df.shape[0])
        trapped_event_df["segment_length"] = segment_length
#         event["segment_length"] = segment_length        
        # Fill in computationally intensive properties of trapped_event_df.
        trapped_event_df = self.fill_in_properties(trapped_event_df)
#         print(trapped_event_df)
        
        # Empty list to be filled with scattered segments.
        scattered_segments_list = []
        
        for event_index, event in trapped_event_df.iterrows():
            
            # Calculate the length of segment 0: 
            # TODO(byron): Build this into a different method?
            mu = self.mean_track_length
            segment_length = np.random.exponential(mu)
            event["segment_length"] = segment_length
            
            # Extract position and center theta from event.
            # Note that this could be weird because rho is not rho center. Maybe just go with original phi and rho and z but change pitch angle
            # Think more about this. 
            center_x, center_y = event["center_x"], event["center_y"]
            rho_pos = event["initial_rho_pos"]
            phi_pos = event["initial_phi_pos"]
            zpos = 0 
            center_theta = event["center_theta"]
            phi_dir = event["initial_phi_dir"] 
            
            # Extract necessary parameters from event. 
            # TODO(byron): Note that it is slightly incorrect to assume the power doesn't change as time passes. 
            energy = event["energy"]
            event_num = event["event_num"]
#             print(event_num)
            segment_radiated_power = event["segment_power"]*2
#             slope = sc.df_dt(event["energy"], self.main_field, segment_radiated_power)
#             event["slope"] = slope
            
            # Extrct 
            # Append segment 0 to scattered_segments_list because segment 0 is trapped by default.
            scattered_segments_list.append(event.values.tolist())
            
            # Begin with trapped beta (segment 0).
            is_trapped = True
            jump_num = 0

            # The infite loop breaks when the trap condition is False or the jump_num exceeds self.jump_num_max.
            while True:
                # Physics happens. This could maybe be wrapped into a different method. 
                
                # First, calculate the segment length, jump size, and delta_pitch_angle.
                # Segment Length: Sampled from exponential. 
                mu = self.mean_track_length
                segment_length = np.random.exponential(mu)
                
                # Jump Size: Sampled from normal dist. 
                mu, sigma = self.jump_size_eV, self.jump_std_eV
                jump_size_eV = np.random.normal(mu, sigma)
                print("jump_size",jump_size_eV)
                
                # Delta Pitch Angle: Sampled from normal dist. 
                mu, sigma = 0, self.pitch_angle_costheta_std 
                rand_float = np.random.normal(mu, sigma) # Necessary to properly distribute angles on a sphere. 
                delta_center_theta = (np.arccos(rand_float)-np.pi/2)*180/np.pi
#                 print(delta_center_theta)
                # Second, calculate new pitch angle and energy.
                # New Pitch Angle: 
                center_theta = center_theta + delta_center_theta

                
                # New Energy: 
#                 print(energy)
                energy = energy-segment_length*segment_radiated_power*JOULE_TO_EV - jump_size_eV
#                 print(energy)
#                 print("Frequency0",sc.energy_to_freq(energy, self.main_field))
#                 print(energy)
                
                beta_position, beta_direction = ([rho_pos, phi_pos, zpos],
                                                 [center_theta, phi_dir])
                
                # Third, construct a scattered, meaning potentially not-trapped, segment df
                scattered_segment_df = self.hardware.construct_untrapped_segment_df(beta_position, beta_direction, energy, event_num)
                
                # Fourth, check to see if the scattered beta is trapped.
                is_trapped = self.hardware.trap_condition(scattered_segment_df)
            
                jump_num += 1
                
                # If the event is not trapped or the max number of jumps has been reached, 
                # we do not want to write the df to the scattered_segments_list.
                if not is_trapped or (jump_num>self.jump_num_max): 
#                     print("Trap Condition : ", is_trapped)
#                     print("Reached jump_num_max : ", (jump_num>self.jump_num_max) )
                    break
                    
                # START HERE WEDS:
                # Now call the function that fills in the df with all the heavy calculations. 
                # The function
                # Lastly, Append to scattered_segments_list if the segment is trapped.
#                 slope = sc.df_dt(event["energy"], self.main_field, segment_radiated_power)
#                 scattered_segment_df["slope"] = slope
#                 print(event_num, jump_num, slope)
                scattered_segment_df["segment_num"] = jump_num
                scattered_segment_df["segment_length"] = segment_length
                scattered_segments_list.append(scattered_segment_df.iloc[0].values.tolist())
        
        # Convert that list to a df. This is much faster than appending to a df. 
#         print(scattered_segments_list)
        scattered_df = pd.DataFrame(scattered_segments_list, columns=trapped_event_df.columns)
#         print(scattered_df)
        scattered_df = self.fill_in_properties(scattered_df)
#         print(scattered_df)
        return scattered_df

    def fill_in_properties(self, incomplete_scattered_segments_df):
        """This method fills in the scattered df with computationally intensive values. 
        
        FILL IN LATER. 
        
        Notes: 
        * May pay off to vectorize this? Going to try. 
        """
         # Constants.
        JOULE_TO_EV = 1/(1.60217733e-19)
        
        df = incomplete_scattered_segments_df.copy()
        
        # Calculate all relevant segment parameters. Order matters here. 
        
        axial_freq = sc.axial_freq(df["energy"], df["center_theta"],df["rho_center"], self.trap_profile)
#         axial_freq = sc.axial_freq(17.8e3, df["center_theta"],df["rho_center"], self.trap_profile)
        avg_cycl_freq = sc.avg_cycl_freq(df["energy"], axial_freq, df["center_theta"],df["rho_center"], self.trap_profile)
#         avg_cycl_freq = sc.avg_cycl_freq(17.8e3, axial_freq, df["center_theta"],df["rho_center"], self.trap_profile)
        zmax = sc.max_zpos(df["center_theta"],df["rho_center"], self.trap_profile)
        mod_index = sc.mod_index(avg_cycl_freq, zmax)
        segment_radiated_power = pc.power_calc(df["center_x"],df["center_y"],
                                               avg_cycl_freq, self.main_field, self.decay_cell_radius)*2
        slope = sc.df_dt(df["energy"], self.main_field, segment_radiated_power)
        energy_stop = df["energy"]- segment_radiated_power*df["segment_length"]*JOULE_TO_EV
#         print("energy_start",df["energy"])
#         print("energy_stop " , energy_stop)
        
        freq_stop = sc.avg_cycl_freq(energy_stop, axial_freq, df["center_theta"],df["rho_center"], self.trap_profile)
#         slope = (freq_stop - avg_cycl_freq)/df["segment_length"]
        
#         slope = sc.avg_cycl_freq(df["energy"], axial_freq, df["center_theta"],df["rho_center"], self.trap_profile)
        segment_power = segment_radiated_power/2

        df["axial_freq"] = axial_freq
        df["avg_cycl_freq"] = avg_cycl_freq
        df["freq_stop"] = freq_stop
        df["zmax"] = zmax
        df["mod_index"] = mod_index
        df["slope"] = slope
        df["segment_power"] = segment_power
        
        return df

## Trying to fix the issues with slopes and make the simulation more physical. 

In [6]:
class Kinematics:
    
    """
    An object containing methods to deal with scattering. 
    
    "Kinematics" : {
        "jump_num" : REDO THIS, 
        "jump_size" : 14,
    },
    """

    def __init__(self, config_dict, hardware):
        
        # Need to address this with inheritance or something: 
        self.hardware = hardware
        
        # Note: Need to get rid of hardware input. Just doing this for now due to the trap profile issue. 
        self.mean_track_length = config_dict["Kinematics"]["mean_track_length"]
        self.jump_size_eV = config_dict["Kinematics"]["jump_size_eV"]
        self.jump_std_eV = config_dict["Kinematics"]["jump_std_eV"]
        self.pitch_angle_costheta_std = config_dict["Kinematics"]["pitch_angle_costheta_std"]
        self.jump_num_max = config_dict["Kinematics"]["jump_num_max"]
        
        # Need this information from "Hardware" as well. 
        self.trap_strength = config_dict["Hardware"]["trap_strength"]
        self.main_field = config_dict["Hardware"]["main_field"]
        self.decay_cell_radius = config_dict["Hardware"]["decay_cell_radius"]
        
        
        # loading trap profile
        # Need to figure out how this will be configurable. For now it is fixed to the default trap profile. 
        # Also need to figure out how this only gets called once for the entire simulation....
        self.trap_profile = hardware.trap_profile
        self.field_strength = lambda r,z : self.trap_profile.field_values((r,0,z))[2]


    def scatter(self, trapped_event_df):
        
        # Constants.
        JOULE_TO_EV = 1/(1.60217733e-19) 
        
#         # TODO: Put the timing in it's own function. 
#         mu = self.mean_track_length
#         segment_length = np.random.exponential(mu,trapped_event_df.shape[0])
        
        
#         event["segment_length"] = segment_length        
        # Fill in computationally intensive properties of trapped_event_df.
        trapped_event_df = self.fill_in_properties(trapped_event_df)
#         print(trapped_event_df)
        
        # Empty list to be filled with scattered segments.
        scattered_segments_list = []
        
        for event_index, event in trapped_event_df.iterrows():
            
            # Calculate length of segment: 
            mu = self.mean_track_length
            segment_length = np.random.exponential(mu)
            event["segment_length"] = segment_length
            
            # Fill the event. 
            event = self.fill_in_properties(event)
#             print(event)

            # Extract position and center theta from event.
            # Note that this could be weird because rho is not rho center. Maybe just go with original phi and rho and z but change pitch angle
            # Think more about this. 
            center_x, center_y = event["center_x"], event["center_y"]
            rho_pos = event["initial_rho_pos"]
            phi_pos = event["initial_phi_pos"]
            zpos = 0 
            center_theta = event["center_theta"]
            phi_dir = event["initial_phi_dir"] 
            
            # Extract necessary parameters from event. 
            # TODO(byron): Note that it is slightly incorrect to assume the power doesn't change as time passes. 
            energy = event["energy"]
            energy_stop = event["energy_stop"]
            event_num = event["event_num"]
#             print(event_num)
            segment_radiated_power = event["segment_power"]*2

            # Append segment 0 to scattered_segments_list because segment 0 is trapped by default.
            scattered_segments_list.append(event.values.tolist())
            
            # Begin with trapped beta (segment 0).
            is_trapped = True
            jump_num = 0

            # The infite loop breaks when the trap condition is False or the jump_num exceeds self.jump_num_max.
            while True:
                scattered_segment = event.copy()
#                 print(scattered_segment)
                # Physics happens. This could maybe be wrapped into a different method. 
                
                # First, calculate the segment length, jump size, and delta_pitch_angle.
#                 Segment Length: Sampled from exponential. 
                mu = self.mean_track_length
                segment_length = np.random.exponential(mu)
                
                # Jump Size: Sampled from normal dist. 
                mu, sigma = self.jump_size_eV, self.jump_std_eV
                jump_size_eV = np.random.normal(mu, sigma)
#                 print("jump_size",jump_size_eV)
                
                # Delta Pitch Angle: Sampled from normal dist. 
                mu, sigma = 0, self.pitch_angle_costheta_std 
                rand_float = np.random.normal(mu, sigma) # Necessary to properly distribute angles on a sphere. 
                delta_center_theta = (np.arccos(rand_float)-np.pi/2)*180/np.pi
#                 print(delta_center_theta)
                # Second, calculate new pitch angle and energy.
                # New Pitch Angle: 
                center_theta = center_theta + delta_center_theta
  
                # New Energy: 
#                 energy = energy-segment_length*segment_radiated_power*JOULE_TO_EV - jump_size_eV
                print("\n", energy, "\n", energy_stop)
                energy = energy_stop - jump_size_eV
                beta_position, beta_direction = ([rho_pos, phi_pos, zpos],
                                                 [center_theta, phi_dir])
                
                # Third, construct a scattered, meaning potentially not-trapped, segment df
                scattered_segment_df = self.hardware.construct_untrapped_segment_df(beta_position, beta_direction, energy, event_num)
                
                # Fourth, check to see if the scattered beta is trapped.
                is_trapped = self.hardware.trap_condition(scattered_segment_df)
            
                jump_num += 1
                
                # If the event is not trapped or the max number of jumps has been reached, 
                # we do not want to write the df to the scattered_segments_list.
                if not is_trapped or (jump_num>self.jump_num_max): 
#                     print("Trap Condition : ", is_trapped)
#                     print("Reached jump_num_max : ", (jump_num>self.jump_num_max) )
                    break
                    
                # START HERE WEDS:
                # Now call the function that fills in the df with all the heavy calculations. 
                # The function
                # Lastly, Append to scattered_segments_list if the segment is trapped.
#                 slope = sc.df_dt(event["energy"], self.main_field, segment_radiated_power)
#                 scattered_segment_df["slope"] = slope
#                 print(event_num, jump_num, slope)
                scattered_segment_df["segment_num"] = jump_num
                scattered_segment_df["segment_length"] = segment_length
                scattered_segment_df = self.fill_in_properties(scattered_segment_df)
#                 print(scattered_segment_df)
                scattered_segments_list.append(scattered_segment_df.iloc[0].values.tolist())
                
                # reset segment_radiated_power
                segment_radiated_power = scattered_segment_df["segment_power"]*2
                energy_stop = scattered_segment_df["energy_stop"]
        # Convert that list to a df. This is much faster than appending to a df. 
#         print(scattered_segments_list)
        scattered_df = pd.DataFrame(scattered_segments_list, columns=trapped_event_df.columns)
#         print(scattered_df)
#         scattered_df = self.fill_in_properties(scattered_df)
#         print(scattered_df)
        return scattered_df

    def fill_in_properties(self, incomplete_scattered_segments_df):
        """This method fills in the scattered df with computationally intensive values. 
        
        FILL IN LATER. 
        
        Notes: 
        * May pay off to vectorize this? Going to try. 
        """
         # Constants.
        JOULE_TO_EV = 1/(1.60217733e-19)
        
        df = incomplete_scattered_segments_df.copy()
        

            
        # Calculate all relevant segment parameters. Order matters here. 
        axial_freq = sc.axial_freq(df["energy"], df["center_theta"],df["rho_center"], self.trap_profile)
        avg_cycl_freq = sc.avg_cycl_freq(df["energy"], df["center_theta"],df["rho_center"], self.trap_profile)
        
        zmax = sc.max_zpos(df["center_theta"],df["rho_center"], self.trap_profile)
        mod_index = sc.mod_index(avg_cycl_freq, zmax)
        segment_radiated_power = pc.power_calc(df["center_x"],df["center_y"],
                                               avg_cycl_freq, self.main_field, self.decay_cell_radius)*2
        slope = sc.df_dt(df["energy"], self.main_field, segment_radiated_power)
        
        energy_stop = df["energy"]- segment_radiated_power*df["segment_length"]*JOULE_TO_EV
        freq_stop = sc.avg_cycl_freq(energy_stop,  df["center_theta"],df["rho_center"], self.trap_profile)
        slope = (freq_stop - avg_cycl_freq)/df["segment_length"]
        
        segment_power = segment_radiated_power/2

        df["axial_freq"] = axial_freq
        df["avg_cycl_freq"] = avg_cycl_freq
        df["freq_stop"] = freq_stop
        df["energy_stop"] = energy_stop
        df["zmax"] = zmax
        df["mod_index"] = mod_index
        df["slope"] = slope
        df["segment_power"] = segment_power
        
        return df

In [17]:
hardware = Hardware(config_dict)

Optimization terminated successfully.
         Current function value: -0.689350
         Iterations: 42
         Function evaluations: 95
Trap width: (-0.04515688705444337,0.04515688705444337)
Maximum Field: 0.6893498199715139
Time to initialize field_strength_interp =  23.910581044000004 



In [31]:

hardware_df = simulation.read_saved_df("Hardware_trapped_events_df") 
hardware_df

In [57]:
kinematics = Kinematics(config_dict, hardware)

In [58]:
kin_df = kinematics.scatter(hardware_df)


 17800.0 
 17791.417284290874
Trapped!

 17791.417284290874 
 0.0    17790.467121
Name: energy_stop, dtype: float64
Trapped!

 0.0    17790.467121
Name: energy_stop, dtype: float64 
 0.0    17777.959865
Name: energy_stop, dtype: float64
Trapped!

 0.0    17777.959865
Name: energy_stop, dtype: float64 
 0.0    17775.712931
Name: energy_stop, dtype: float64
Trapped!

 0.0    17775.712931
Name: energy_stop, dtype: float64 
 0.0    17772.062971
Name: energy_stop, dtype: float64
Trapped!

 17800.0 
 17780.270810037568
Trapped!

 17780.270810037568 
 1.0    17758.817639
Name: energy_stop, dtype: float64
Trapped!

 1.0    17758.817639
Name: energy_stop, dtype: float64 
 1.0    17728.746514
Name: energy_stop, dtype: float64
Trapped!

 1.0    17728.746514
Name: energy_stop, dtype: float64 
 1.0    17728.250531
Name: energy_stop, dtype: float64
Trapped!

 1.0    17728.250531
Name: energy_stop, dtype: float64 
 1.0    17665.591766
Name: energy_stop, dtype: float64
Trapped!

 17800.0 
 17796.5127

In [34]:
kin_df

,energy,initial_rho_pos,initial_phi_pos,initial_zpos,initial_pitch_angle,initial_phi_dir,center_theta,initial_field,initial_radius,center_x,center_y,rho_center,trapped_initial_pitch_angle,max_radius,avg_cycl_freq,zmax,axial_freq,mod_index,segment_power,slope,segment_length,band_power,band_num,segment_num,event_num,freq_stop
0,17800.000000,0.004358,259.430879,0.001022,89.917568,21.366756,89.801383,0.688265,0.000659,0.004118,0.000614,0.004163,87.704337,0.000659,1.861779e+10,0.001124,3.780312e+07,0.253258,1.996181e-16,8.773330e+07,0.003748,NaN,NaN,0.0,0.0,1.861812e+10
1,17762.880711,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000659,0.004118,0.000613,0.004163,87.697234,0.000659,1.861910e+10,0.001124,3.776599e+07,0.253309,1.992078e-16,8.756988e+07,0.014896,NaN,NaN,1.0,0.0,1.862040e+10
2,17727.304353,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000658,0.004118,0.000613,0.004163,87.697228,0.000658,1.862035e+10,0.001124,3.773036e+07,0.253358,1.988146e-16,8.740851e+07,0.014277,NaN,NaN,2.0,0.0,1.862160e+10
3,17670.462642,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000657,0.004118,0.000612,0.004164,87.697218,0.000657,1.862235e+10,0.001124,3.767337e+07,0.253436,1.981863e-16,8.715448e+07,0.022811,NaN,NaN,3.0,0.0,1.862434e+10
4,17670.276653,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000657,0.004118,0.000612,0.004164,87.697218,0.000657,1.862236e+10,0.001124,3.767317e+07,0.253436,1.981843e-16,8.714437e+07,0.000075,NaN,NaN,4.0,0.0,1.862236e+10
5,17800.000000,0.003144,32.461799,0.001379,88.315167,52.864956,88.299992,0.688296,0.000659,0.002619,0.000398,0.002649,87.755869,0.000659,1.863039e+10,0.018718,1.298257e+07,4.226893,3.375006e-16,1.484351e+08,0.003298,NaN,NaN,0.0,1.0,1.863088e+10
6,17779.584024,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000659,0.002619,0.000398,0.002649,87.744450,0.000659,1.863111e+10,0.018718,1.297569e+07,4.227324,3.370997e-16,1.482721e+08,0.004846,NaN,NaN,1.0,1.0,1.863183e+10
7,17776.921966,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000659,0.002619,0.000398,0.002649,87.744449,0.000659,1.863120e+10,0.018718,1.297479e+07,4.227380,3.370473e-16,1.482456e+08,0.000632,NaN,NaN,2.0,1.0,1.863129e+10
8,17772.926089,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000659,0.002619,0.000398,0.002649,87.744448,0.000659,1.863134e+10,0.018718,1.297345e+07,4.227465,3.369688e-16,1.482136e+08,0.000948,NaN,NaN,3.0,1.0,1.863148e+10
9,17768.852140,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000658,0.002619,0.000397,0.002649,87.744447,0.000658,1.863149e+10,0.018718,1.297208e+07,4.227550,3.368887e-16,1.481807e+08,0.000967,NaN,NaN,4.0,1.0,1.863163e+10


In [59]:
print(kin_df["avg_cycl_freq"],kin_df["freq_stop"])

0     1.861779e+10
1     1.861809e+10
2     1.861812e+10
3     1.861856e+10
4     1.861864e+10
5     1.863039e+10
6     1.863108e+10
7     1.863184e+10
8     1.863290e+10
9     1.863292e+10
10    1.861983e+10
11    1.861995e+10
12    1.862014e+10
13    1.862020e+10
14    1.862094e+10
Name: avg_cycl_freq, dtype: float64 0     1.861809e+10
1     1.861812e+10
2     1.861856e+10
3     1.861864e+10
4     1.861877e+10
5     1.863108e+10
6     1.863184e+10
7     1.863290e+10
8     1.863292e+10
9     1.863512e+10
10    1.861995e+10
11    1.862014e+10
12    1.862020e+10
13    1.862094e+10
14    1.862126e+10
Name: freq_stop, dtype: float64


In [60]:
print(kin_df["energy"],kin_df["energy_stop"])

0     17800.000000
1     17791.417284
2     17790.467121
3     17777.959865
4     17775.712931
5     17800.000000
6     17780.270810
7     17758.817639
8     17728.746514
9     17728.250531
10    17800.000000
11    17796.512781
12    17791.153687
13    17789.540227
14    17768.512047
Name: energy, dtype: float64 0     17791.417284
1     17790.467121
2     17777.959865
3     17775.712931
4     17772.062971
5     17780.270810
6     17758.817639
7     17728.746514
8     17728.250531
9     17665.591766
10    17796.512781
11    17791.153687
12    17789.540227
13    17768.512047
14    17759.427965
Name: energy_stop, dtype: float64


In [108]:
 # Constants.
JOULE_TO_EV = 1/(1.60217733e-19)

trapped_event_df = hardware_df

mu = kinematics.mean_track_length
mu = .3
segment_length = np.random.exponential(mu,trapped_event_df.shape[0])

# Fill in what you need for the trapped_event_df (axial_freq, avg_cycl_freq, segment_radiated_power)
axial_freq = sc.axial_freq(trapped_event_df["energy"], trapped_event_df["center_theta"],
                           trapped_event_df["rho_center"], hardware.trap_profile)
avg_cycl_freq = sc.avg_cycl_freq(trapped_event_df["energy"], axial_freq, trapped_event_df["center_theta"],
                                 trapped_event_df["rho_center"], hardware.trap_profile)

# Sanity check: Yes, avge_cycl_freq works at least to first order.
# print(sc.energy_to_freq(trapped_event_df["energy"],hardware.main_field ))
# print(avg_cycl_freq )

segment_radiated_power = pc.power_calc(trapped_event_df["center_x"],trapped_event_df["center_y"],
                                       avg_cycl_freq, hardware.main_field, hardware.decay_cell_radius)*2
slope = sc.df_dt(trapped_event_df["energy"], hardware.main_field, segment_radiated_power)

print(slope)
energy_loss_eV = segment_radiated_power*segment_length*JOULE_TO_EV
energy_stop = trapped_event_df["energy"]- energy_loss_eV
energy_start = trapped_event_df["energy"]



## Now calculate the energy loss according to the slope calculation: 

E1 = sc.freq_to_energy(avg_cycl_freq, hardware.main_field)
E2 = sc.freq_to_energy(avg_cycl_freq+ slope*segment_length, hardware.main_field)
# print(E1,E2)
print("METHOD 2: energy_loss_eV \n",E1-E2)
print("METHOD 2: frequency change \n", slope*segment_length)
#         print(energy_stop)
# This uses the original axial_freq which may be problematic...
axial_freq_stop = sc.axial_freq(energy_stop, trapped_event_df["center_theta"],
                           trapped_event_df["rho_center"], hardware.trap_profile)
freq_stop = sc.avg_cycl_freq(energy_stop, axial_freq_stop, trapped_event_df["center_theta"],
                             trapped_event_df["rho_center"], hardware.trap_profile)
print("\nMETHOD 1: energy_loss_eV \n", energy_start-energy_stop)
print("METHOD 1: frequency change \n", freq_stop-avg_cycl_freq)


E1_alt = sc.freq_to_energy(avg_cycl_freq, hardware.main_field)
E2_alt = sc.freq_to_energy(freq_stop, hardware.main_field)
print("\n METHOD 3: energy_loss_eV \n", E1_alt-E2_alt)
# print(avg_cycl_freq, freq_stop)
slope = (freq_stop - avg_cycl_freq)/segment_length
print(slope)
# print("power1",segment_radiated_power)
# print("power2",pc.power_calc(trapped_event_df["center_x"],trapped_event_df["center_y"],
#                                        freq_stop, hardware.main_field, hardware.decay_cell_radius)*2)
segment_power = segment_radiated_power/2

trapped_event_df["axial_freq"] = axial_freq
trapped_event_df["avg_cycl_freq"] = avg_cycl_freq
#         trapped_event_df["zmax"] = zmax
#         trapped_event_df["mod_index"] = mod_index
trapped_event_df["slope"] = slope
trapped_event_df["segment_power"] = segment_power
trapped_event_df["segment_length"] = segment_length
# print(trapped_event_df)

0    8.502182e+07
1    9.294933e+07
Name: energy, dtype: float64
METHOD 2: energy_loss_eV 
 0    2511.184519
1     538.103465
Name: energy, dtype: float64
METHOD 2: frequency change 
 0    8.874135e+07
1    1.895690e+07
Name: energy, dtype: float64

METHOD 1: energy_loss_eV 
 0    2517.829079
1     537.857910
Name: energy, dtype: float64
METHOD 1: frequency change 
 [89071747.33607864 18962219.614254  ]

 METHOD 3: energy_loss_eV 
 [2520.48955586  538.25428694]
[85338373.58875048 92975409.20312008]


In [114]:
# Constants.
JOULE_TO_EV = 1/(1.60217733e-19)

trapped_event_df = hardware_df

mu = kinematics.mean_track_length
mu = .3
segment_length = np.random.exponential(mu,trapped_event_df.shape[0])


segment_radiated_power = pc.power_calc(trapped_event_df["center_x"],trapped_event_df["center_y"],
                                       avg_cycl_freq, hardware.main_field, hardware.decay_cell_radius)*2
slope = sc.df_dt(trapped_event_df["energy"], hardware.main_field, segment_radiated_power)

print(slope)
energy_loss_eV = segment_radiated_power*segment_length*JOULE_TO_EV
energy_stop = trapped_event_df["energy"]- energy_loss_eV
energy_start = trapped_event_df["energy"]

freq_start = sc.avg_cycl_freq(energy_start, trapped_event_df["center_theta"],
                           trapped_event_df["rho_center"], hardware.trap_profile)
freq_stop = sc.avg_cycl_freq(energy_stop, trapped_event_df["center_theta"],
                           trapped_event_df["rho_center"], hardware.trap_profile)

slope = (freq_stop-freq_start)/segment_length
print(slope)

0    8.502182e+07
1    9.294933e+07
Name: energy, dtype: float64
37634424.228067435
37634424.228067435
25571002.998161945
37505200.746231414
37505200.746231414
24441759.034462817
[84952675.30312274 93164721.13201658]


##  This issue seems to be eith avg_cycl_freq: 

Notes: 
* In the theta = 90 case, things align below. 
* Axial Freq can't be left constant if you change energy!

In [106]:
energy = np.arange(100e3,400e3, 50e3 )
center_theta = trapped_event_df["center_theta"][0]
rho_center = trapped_event_df["rho_center"][0]

axial_freq = sc.axial_freq(energy, center_theta, rho_center, hardware.trap_profile)
cyc_freq_1 = sc.avg_cycl_freq(energy, axial_freq, center_theta,
                                 rho_center, hardware.trap_profile)
cyc_freq_2 = sc.energy_to_freq(energy, hardware.main_field)

plt.plot(energy, cyc_freq_1, energy, cyc_freq_2)
print(cyc_freq_1,cyc_freq_2)

[1.61131941e+10 1.48943424e+10 1.38469187e+10 1.29371331e+10
 1.21395283e+10 1.14345606e+10] [1.61302202e+10 1.49100806e+10 1.38615501e+10 1.29508031e+10
 1.21523555e+10 1.14466430e+10]


# BandBuilder Block 

In [7]:
class BandBuilder:
    
    """
    An object containing methods to construct bands from segments. 
    
    "BandBuilder" : {
        "sideband_num": 5,
        "frac_total_segment_power_cut" : 0.01, 

    """

    def __init__(self, config_dict):
        
        self.sideband_num = config_dict["BandBuilder"]["sideband_num"]
        self.frac_total_segment_power_cut = config_dict["BandBuilder"]["frac_total_segment_power_cut"]
        
    def bandbuilder(self, segments_df):
        
        total_band_num = self.sideband_num*2+1
        
        band_list = []
#         print(segments_df.iloc[:,'energy'])
#         for row in segments_df.itertuples(): 
        for segment_index, row in segments_df.iterrows(): 
            
            sideband_magnitudes = sc.sideband_calc(row["avg_cycl_freq"], row["axial_freq"], row["zmax"], num_sidebands = self.sideband_num)[0]
#             print(sideband_magnitudes)
            for i,band_num in enumerate(range(-self.sideband_num, self.sideband_num+1)): 
                
                if sideband_magnitudes[i][1] < self.frac_total_segment_power_cut: 
                    continue
                else: 
                    # copy segment in order to fill in band specific values 
                    row_copy = row.copy()
                    
                    # fill in new avg_cycl_freq, band_power, band_num
                    row_copy["avg_cycl_freq"] = sideband_magnitudes[i][0]
                    row_copy["band_power"] = sideband_magnitudes[i][1]*row.segment_power
                    row_copy["band_num"] = band_num
#                     print(row_copy["band_num"], sideband_magnitudes[i][1])
                    # append to band_list, as it's better to grow a list than a df
                    band_list.append(row_copy.tolist())
#         print(band_list)
        bands_df = pd.DataFrame(band_list, columns=segments_df.columns)
#         bands_df.columns = segments_df.columns
#         print(bands_df)
        return bands_df

    

# TrackBuilder Block 

In [8]:
class TrackBuilder:
    
    """
    An object containing methods to construct bands from segments. 
    
    "TrackBuilder" : {
        "trackbuilder_freqbw_max" : 19.3e9,
        "trackbuilder_freqbw_min" : 17.6e9,
        "run_length" : 1, 
        "decay_rate" : 1e3
    },


    """

    def __init__(self, config_dict):
        
        self.run_length = config_dict["TrackBuilder"]["run_length"]
        self.decay_rate = config_dict["TrackBuilder"]["decay_rate"]
        self.events_to_simulate = config_dict["Physics"]["events_to_simulate"]
    
    def trackbuilder(self, bands_df):
        
        # Need to add in BW cut, and timing...
        # The timing is actually tricky. Need to be careful that the event rate is physical otherwise this won't be able to tell us what fraction we're seeing. 
        # May want to look at sc.cycl_prob and sc.sph_prob 
        
        # add time/freq start/stop   
        tracks_df = bands_df.copy()
        tracks_df["time_start"] = np.NaN
        tracks_df["time_stop"] = np.NaN
    
        tracks_df["freq_start"] = bands_df["avg_cycl_freq"]
        tracks_df["freq_stop"] = bands_df["slope"]*bands_df["segment_length"]+bands_df["avg_cycl_freq"]

        # dealing with timing of the events. 
        # for now just put all events in the window... need to think about this. 
        trapped_event_start_times = np.random.uniform(0,self.run_length,self.events_to_simulate)
        
        # iterate through the segment zeros and fill in start times.
        
        for index, row in bands_df[bands_df["segment_num"]==0.0].iterrows():
#             print(index)
            event_num = int(tracks_df["event_num"][index])
#             print(event_num)
            tracks_df["time_start"][index] = trapped_event_start_times[event_num]
        
        for event in range(0,self.events_to_simulate): 
            
            # find max segment_num for each event
            segment_num_max = int(bands_df[bands_df["event_num"]==event]["segment_num"].max())

            for segment in range(1,segment_num_max+1):

                fill_condition = ((tracks_df["event_num"]== float(event)) & (tracks_df["segment_num"] == segment))
                previous_time_condition = ((tracks_df["event_num"]== event) & (tracks_df["segment_num"] == segment-1) 
                                           & (tracks_df["band_num"] == 0.0) )
#                 print("previous_time_condition : ", previous_time_condition)
                previous_segment_time_start = tracks_df[previous_time_condition]["time_start"].iloc[0]
                previous_segment_length = tracks_df[previous_time_condition]["segment_length"].iloc[0]

                for index, row in tracks_df[fill_condition].iterrows(): 
                    tracks_df["time_start"][index] = previous_segment_time_start + previous_segment_length
                    

        tracks_df["time_stop"] = tracks_df["time_start"]+ tracks_df["segment_length"]

        tracks_df = tracks_df.drop(columns = ["initial_rho_pos","initial_zpos","initial_pitch_angle", "trapped_initial_pitch_angle",
                              "initial_phi_dir", "center_theta", "initial_field", "initial_radius",
                              "center_x", "center_y", "rho_center", "max_radius", "zmax", "mod_index", "avg_cycl_freq", "axial_freq"])
        
        return tracks_df

                
        

# Simulation Block: 

This is the "main()" that calls all the other blocks. 

In [9]:
# Trying something new here with the scattering so that this makes more sense. 

class Configure_Simulation:
    
    """
    A class that runs a simulation according to the config_dict. It will also save the df's at different points to csv files.  
    
    "Simulation" : {
        "check_for_existing_sim_data" : true,
        "simulation_results_dir" : "Example_Simulation_dfs",
        "simulation_results_file_prefix" : "Not sure I need this"
    """

    def __init__(self, config_dict):
        
        self.simulation_results_dir = config_dict["Configure_Simulation"]["simulation_results_dir"]


    def run(self):
        
        # initialize all simulation blocks: 
        # may be useful to make these attributes of config for ease? 
        phys = Physics(config_dict)
        hardware = Hardware(config_dict)
        kinematics = Kinematics(config_dict,hardware)
        bandbuilder = BandBuilder(config_dict)
        trackbuilder = TrackBuilder(config_dict)
        
        # Generate a base set of events: 
        
        event_num = 0 
        
        while event_num < phys.events_to_simulate:

            print("\n Event: {}/{}...\n".format(event_num, phys.events_to_simulate-1))

            # generate trapped beta
            is_trapped = False
            
            while not is_trapped:
                
                initial_position, initial_direction = phys.generate_beta_position_direction()
                energy = phys.generate_beta_energy()

                single_segment_df = hardware.construct_untrapped_segment_df(initial_position, initial_direction, energy, event_num)
                
                # Will set is_trapped to True if trap condition is met. 
#                 is_trapped = hardware.trap_condition(single_segment_df, event_num)
                is_trapped = hardware.trap_condition(single_segment_df)
            # add zmax, avg_cycl_freq, axial_freq to df now that we have a trapped beta
#             single_trapped_segment_df = hardware.construct_trapped_segment_df(single_segment_df, event_num)   
        
            if event_num == 0: 
                trapped_event_df = single_segment_df
            else: 
                
                trapped_event_df = trapped_event_df.append(single_segment_df, ignore_index=True)
                
            event_num +=1  

#         print(trapped_event_df)    
        # save the trapped events df:
        self.save_df(trapped_event_df,"Hardware_trapped_events_df" )
        
        # add scattering.
        segments_df = kinematics.scatter(trapped_event_df)
        
        # save the scattered segments df:
        self.save_df(segments_df,"Kinematics_segments_df" )
        
        # apply band builder. 
        bands_df = bandbuilder.bandbuilder(segments_df)
        
        # save the scattered segments df:
        self.save_df(bands_df,"BandBuilder_bands_df" )
        
        # apply track builder. 
        bands_df = trackbuilder.trackbuilder(bands_df)
        
        # save the scattered segments df:
        self.save_df(bands_df,"TrackBuilder_tracks_df" )
        
        return None
    
    
    def save_df(self, df, filename): 
        
        # Making the dir if it doesn't exist would be a nice touch. 
        df.to_csv('{}/{}/{}.csv'.format(os.getcwd(),self.simulation_results_dir,filename))

        return 0
    
    def read_saved_df(self,filename):
        
        df = pd.read_csv('{}/{}/{}.csv'.format(os.getcwd(),self.simulation_results_dir,filename),index_col=[0])
        
        return df

In [10]:
simulation = Configure_Simulation(config_dict)
simulation.run()


Optimization terminated successfully.
         Current function value: -0.689350
         Iterations: 42
         Function evaluations: 95
Trap width: (-0.04515688705444337,0.04515688705444337)
Maximum Field: 0.6893498199715139
Time to initialize field_strength_interp =  25.344861463 


 Event: 0/2...

Not Trapped: Pitch angle too small.
Not Trapped: Pitch angle too small.
Not Trapped: Pitch angle too small.
Trapped!

 Event: 1/2...

Trapped!

 Event: 2/2...

Trapped!

 17800.0 
 17788.074567976775
Trapped!

 17788.074567976775 
 0.0    17758.378959
Name: energy_stop, dtype: float64
Trapped!

 0.0    17758.378959
Name: energy_stop, dtype: float64 
 0.0    17750.508301
Name: energy_stop, dtype: float64
Trapped!

 0.0    17750.508301
Name: energy_stop, dtype: float64 
 0.0    17739.718736
Name: energy_stop, dtype: float64
Trapped!

 0.0    17739.718736
Name: energy_stop, dtype: float64 
 0.0    17729.744628
Name: energy_stop, dtype: float64
Trapped!

 17800.0 
 17790.748370006422
Trapped

# Understanding the outputs of each Block: 

8/3/21: Going through each block and making sure the outputs make physical sense and are working. 

**Physics:**

* Need to make sure the phi variable is being randomized correctly. 
* Currently only works for monoenergetic electrons but that's fine for now. 

**Hardware:** 

* intial_rho_pos and rho_center make sense. Their difference is always less than the initial_radius. 
* avg_cycl_freq, zmax, axial_freq, mod_index 
    * None of these work yet. 
    * sc.avg_cyc_freq and axial_freq takes a very long time to run. Going to be too slow...  
        * Kris had the timestep (dtime) being used to find the average field set very small (10e-12), I changed it to 10e-9 for now. But will need to revisit this, as it's **not nearly as accurate now**. As an illustration, the avg_cyc_freq changed fractionally by 10e-5 and axial_freq changed fractionally by .2 when changing dtime from 1e-9 to 1e-11. So need to be more careful with axial_freq. Could their be a decent approximation that can be done instead of this timestep method? Maybe a table and a fit or something, and then using Kris's white paper results to map between fields. Should maybe put effort into the alternative way to calculate axial_freq that Kris outlines in a whitepaper.
* Do need to implement some frequency cut in the hardware block for telling you that you aren't going to see any tracks in your visable BW, but going to wait on this until it becomes clearer how it should be implemented. 

**Kinematics:**

* Right now this doesn't work well at all. 
    * It needs to account for the slope times the track length, and for the jump.
* For now: 
    * Calculate the track lengths from a mean_lifetime or from sc.collision_rate. Note that you need to use Pa's in the collision rate function. This is more or less working above. 
    * The tricky thing is modelling how many tracks and how large the jumps should be. Look to Ali and Christine's thesis for guidance. We need event reconstruction before we can get at these distributions. Additionally how do you decide how to model the scattering's effect on the zmax and thus visable power? One interesting thing is that we typically see many visable tracks in a row, which doesn't make sense niavely in that so few of our tracks are visable given a random distribution of pitch angles. Meaning the scattering must not affect the pitch angle a ton? Once you have the rest of this working ok, then meet with Christine to discuss this. 
    * For now, account for the slope, make a fixed jump height of 14 eV AND make a fixed number of jumps. 


## Hardware Output: Set of 5 Trapped Events:

* An exhaustive list of attributes are saved for each event.

In [29]:
hardware_df = simulation.read_saved_df("Hardware_trapped_events_df") 
hardware_df

,energy,initial_rho_pos,initial_phi_pos,initial_zpos,initial_pitch_angle,initial_phi_dir,center_theta,initial_field,initial_radius,center_x,center_y,rho_center,trapped_initial_pitch_angle,max_radius,avg_cycl_freq,zmax,axial_freq,mod_index,segment_power,slope,segment_length,band_power,band_num,segment_num,event_num
0,17800.0,0.004358,259.430879,0.001022,89.917568,21.366756,89.801383,0.688265,0.000659,0.004118,0.000614,0.004163,87.704337,0.000659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0
1,17800.0,0.003144,32.461799,0.001379,88.315167,52.864956,88.299992,0.688296,0.000659,0.002619,0.000398,0.002649,87.755869,0.000659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,1
2,17800.0,0.003710,109.191654,-0.001406,89.181332,228.650447,89.144316,0.688286,0.000659,0.004205,-0.000436,0.004227,87.708173,0.000659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,2


## Kinematics Output: Set of 15 Segments:

* This block scatters the trapped events creating segments of a certain length and starting frequency/energy associated with each trapped event. 
    * For illustration, each trapped event is scattered twice and each segment lasts 1 s. 
* At this stage each segment has a total power and length but start and stop times have not yet been assigned. 

In [30]:
simulation.read_saved_df("Kinematics_segments_df") 

,energy,initial_rho_pos,initial_phi_pos,initial_zpos,initial_pitch_angle,initial_phi_dir,center_theta,initial_field,initial_radius,center_x,center_y,rho_center,trapped_initial_pitch_angle,max_radius,avg_cycl_freq,zmax,axial_freq,mod_index,segment_power,slope,segment_length,band_power,band_num,segment_num,event_num,freq_stop
0,17800.000000,0.004358,259.430879,0.001022,89.917568,21.366756,89.801383,0.688265,0.000659,0.004118,0.000614,0.004163,87.704337,0.000659,1.861779e+10,0.001124,3.780312e+07,0.253258,1.996181e-16,8.773317e+07,0.003420,NaN,NaN,0.0,0.0,1.861809e+10
1,17789.767957,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000659,0.004118,0.000614,0.004163,87.697238,0.000659,1.861815e+10,0.001124,3.779289e+07,0.253272,1.995051e-16,8.768716e+07,0.004106,NaN,NaN,1.0,0.0,1.861851e+10
2,17771.889378,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000659,0.004118,0.000614,0.004163,87.697235,0.000659,1.861878e+10,0.001124,3.777500e+07,0.253297,1.993074e-16,8.760749e+07,0.007175,NaN,NaN,2.0,0.0,1.861941e+10
3,17771.139724,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000659,0.004118,0.000614,0.004163,87.697235,0.000659,1.861880e+10,0.001124,3.777425e+07,0.253298,1.992991e-16,8.760126e+07,0.000301,NaN,NaN,3.0,0.0,1.861883e+10
4,17744.146801,0.004358,259.430879,0.000000,89.801383,21.366756,89.801383,0.688258,0.000658,0.004118,0.000613,0.004163,87.697231,0.000658,1.861975e+10,0.001124,3.774724e+07,0.253335,1.990008e-16,8.748337e+07,0.010833,NaN,NaN,4.0,0.0,1.862070e+10
5,17800.000000,0.003144,32.461799,0.001379,88.315167,52.864956,88.299992,0.688296,0.000659,0.002619,0.000398,0.002649,87.755869,0.000659,1.863039e+10,0.018718,1.298257e+07,4.226893,3.375006e-16,1.484363e+08,0.004265,NaN,NaN,0.0,1.0,1.863102e+10
6,17797.314131,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000659,0.002619,0.000398,0.002649,87.744456,0.000659,1.863048e+10,0.018718,1.298166e+07,4.226951,3.374482e-16,1.484104e+08,0.000638,NaN,NaN,1.0,1.0,1.863058e+10
7,17770.512266,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000658,0.002619,0.000398,0.002649,87.744447,0.000658,1.863143e+10,0.018718,1.297263e+07,4.227515,3.369214e-16,1.482005e+08,0.006362,NaN,NaN,2.0,1.0,1.863237e+10
8,17716.364810,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000657,0.002620,0.000397,0.002650,87.744431,0.000657,1.863333e+10,0.018717,1.295437e+07,4.228656,3.358571e-16,1.477702e+08,0.012852,NaN,NaN,3.0,1.0,1.863523e+10
9,17715.839757,0.003144,32.461799,0.000000,88.299992,52.864956,88.299992,0.688284,0.000657,0.002620,0.000397,0.002650,87.744430,0.000657,1.863335e+10,0.018717,1.295420e+07,4.228667,3.358468e-16,1.477511e+08,0.000125,NaN,NaN,4.0,1.0,1.863337e+10


## TrackBuilder Output: Set of 45 Tracks:

* This block takes each segment and uses the zmax/mod_index to construct an appropriate number of bands for each segment.
    * To illustrate how it works, I manually made every segment contain one set of sidebands (3 bands per segment). 
* It then spreads the segment_power across the different bands. 
* An intermediate output ("bands_df") contains exhaustive band information. 
* The block assigns start times to events according to configurable parameters. 
* Then the block cuts all attributes not necessary for constructing a fake spec file via the DAQ block ("tracks_df"). 
* The tracks_df can then be input into the DAQ block. 


## Trouble shooting the track start frequencies. 

In [52]:
hardware = Hardware(config_dict)

Optimization terminated successfully.
         Current function value: -0.689350
         Iterations: 42
         Function evaluations: 95
Trap width: (-0.04515688705444337,0.04515688705444337)
Maximum Field: 0.6893498199715139
Time to initialize field_strength_interp =  26.28644017900001 



In [101]:
track_df = simulation.read_saved_df("TrackBuilder_tracks_df") 
band_0 = track_df[track_df["band_num"] == 0]
band_0

,energy,initial_phi_pos,segment_power,slope,segment_length,band_power,band_num,segment_num,event_num,time_start,time_stop,freq_start,freq_stop
2,17800.000000,119.725914,1.551674e-16,6.826840e+07,0.010317,1.247015e-16,0.0,0.0,0.0,0.207838,0.218154,1.861875e+10,1.861945e+10
7,17779.301536,119.725914,1.550208e-16,6.820923e+07,0.010686,1.245755e-16,0.0,1.0,0.0,0.218154,0.228841,1.861948e+10,1.862021e+10
12,17770.261099,119.725914,1.549576e-16,6.818377e+07,0.004667,1.245211e-16,0.0,2.0,0.0,0.228841,0.233508,1.861979e+10,1.862011e+10
17,17768.822483,119.725914,1.549476e-16,6.817972e+07,0.000743,1.245125e-16,0.0,3.0,0.0,0.233508,0.234251,1.861985e+10,1.861990e+10
22,17751.568018,119.725914,1.548270e-16,6.813110e+07,0.008908,1.244086e-16,0.0,4.0,0.0,0.234251,0.243159,1.862045e+10,1.862106e+10
27,17746.970871,119.725914,1.547948e-16,6.811814e+07,0.002373,1.243810e-16,0.0,5.0,0.0,0.243159,0.245532,1.862062e+10,1.862078e+10
32,17737.867373,119.725914,1.547312e-16,6.809248e+07,0.004700,1.243262e-16,0.0,6.0,0.0,0.245532,0.250232,1.862094e+10,1.862126e+10
37,17728.671361,119.725914,1.546669e-16,6.806655e+07,0.004748,1.242708e-16,0.0,7.0,0.0,0.250232,0.254980,1.862126e+10,1.862158e+10
42,17800.000000,136.104322,2.956955e-16,1.300960e+08,0.003718,5.108500e-17,0.0,0.0,1.0,0.461489,0.465206,1.863802e+10,1.863850e+10
47,17799.732475,136.104322,2.956915e-16,1.300944e+08,0.000072,5.108514e-17,0.0,1.0,1.0,0.465206,0.465279,1.863803e+10,1.863804e+10


In [58]:
# Slopes makes sense. 
print(band_0["segment_length"]* band_0["slope"])
print(band_0["freq_stop"]- band_0["freq_start"])

2     1.117251e+05
7     4.846393e+05
12    1.448568e+05
17    7.383737e+05
22    1.707627e+06
27    1.343602e+05
32    2.611274e+06
37    5.429986e+05
42    1.557666e+06
47    1.119739e+05
52    5.511456e+05
57    1.555688e+05
62    1.836107e+06
67    3.069858e+06
72    7.700159e+05
77    6.570272e+04
dtype: float64
2     1.117251e+05
7     4.846393e+05
12    1.448568e+05
17    7.383737e+05
22    1.707627e+06
27    1.343602e+05
32    2.611274e+06
37    5.429986e+05
42    1.557666e+06
47    1.119739e+05
52    5.511456e+05
57    1.555688e+05
62    1.836107e+06
67    3.069858e+06
72    7.700159e+05
77    6.570272e+04
dtype: float64


In [ ]:
# This is a clue. 

In [59]:
slope = sc.df_dt(band_0["energy"], hardware.main_field, band_0["segment_power"]*2)
print(slope)
print(band_0["slope"])

2     1.370332e+08
7     1.369466e+08
12    1.369208e+08
17    1.367891e+08
22    1.364844e+08
27    1.364604e+08
32    1.359939e+08
37    1.358968e+08
42    2.050064e+08
47    2.049737e+08
52    2.048098e+08
57    2.047635e+08
62    2.042175e+08
67    2.033039e+08
72    2.030747e+08
77    2.030551e+08
dtype: float64
2     1.370332e+08
7     1.370332e+08
12    1.370332e+08
17    1.370332e+08
22    1.370332e+08
27    1.370332e+08
32    1.370332e+08
37    1.370332e+08
42    2.050064e+08
47    2.050064e+08
52    2.050064e+08
57    2.050064e+08
62    2.050064e+08
67    2.050064e+08
72    2.050064e+08
77    2.050064e+08
Name: slope, dtype: float64


In [215]:
band_df = simulation.read_saved_df("BandBuilder_bands_df") 

In [216]:
band_0_df = band_df[band_df["band_num"] == 0]
band_0_df

,energy,initial_rho_pos,initial_phi_pos,initial_zpos,initial_pitch_angle,initial_phi_dir,center_theta,initial_field,initial_radius,center_x,center_y,rho_center,trapped_initial_pitch_angle,max_radius,avg_cycl_freq,zmax,axial_freq,mod_index,segment_power,slope,segment_length,band_power,band_num,segment_num,event_num,freq_stop
2,17800.000000,0.001614,248.700400,0.001055,89.222916,209.374428,89.204499,0.688310,0.000659,0.001937,-0.000574,0.002021,87.763871,0.000659,1.862057e+10,0.005266,3.121168e+07,1.187244,3.993855e-16,1.755659e+08,0.006402,2.705708e-16,0.0,0.0,0.0,1.862169e+10
7,17791.636269,0.001614,248.700400,0.000000,89.204499,209.374428,89.204499,0.688304,0.000659,0.001937,-0.000574,0.002021,87.757409,0.000659,1.862086e+10,0.005266,3.120460e+07,1.187304,3.992151e-16,NaN,0.000000,2.704435e-16,0.0,1.0,0.0,1.862086e+10
12,17653.562593,0.001614,248.700400,0.000000,89.204499,209.374428,89.204499,0.688304,0.000656,0.001936,-0.000572,0.002019,87.757442,0.000656,1.862573e+10,0.005266,3.108752e+07,1.188301,3.964038e-16,NaN,0.000000,2.683434e-16,0.0,2.0,0.0,1.862573e+10
17,17641.801236,0.001614,248.700400,0.000000,89.204499,209.374428,89.204499,0.688304,0.000656,0.001936,-0.000572,0.002019,87.757445,0.000656,1.862614e+10,0.005266,3.107753e+07,1.188386,3.961643e-16,NaN,0.000000,2.681646e-16,0.0,3.0,0.0,1.862614e+10
22,17637.234441,0.001614,248.700400,0.000000,89.204499,209.374428,89.204499,0.688304,0.000656,0.001936,-0.000572,0.002019,87.757446,0.000656,1.862630e+10,0.005266,3.107365e+07,1.188419,3.960712e-16,NaN,0.000000,2.680951e-16,0.0,4.0,0.0,1.862630e+10
27,17623.213660,0.001614,248.700400,0.000000,89.204499,209.374428,89.204499,0.688304,0.000656,0.001936,-0.000572,0.002018,87.757449,0.000656,1.862680e+10,0.005266,3.106172e+07,1.188520,3.957856e-16,NaN,0.000000,2.678819e-16,0.0,5.0,0.0,1.862680e+10
32,17612.758516,0.001614,248.700400,0.000000,89.204499,209.374428,89.204499,0.688304,0.000656,0.001936,-0.000571,0.002018,87.757452,0.000656,1.862716e+10,0.005266,3.105283e+07,1.188596,3.955726e-16,NaN,0.000000,2.677230e-16,0.0,6.0,0.0,1.862716e+10
37,17800.000000,0.004787,237.973349,-0.004582,89.794149,77.720481,89.205319,0.688380,0.000659,0.004143,0.000140,0.004145,87.829607,0.000659,1.861956e+10,0.004766,3.456589e+07,1.074316,2.006316e-16,8.818697e+07,0.003325,1.467859e-16,0.0,0.0,1.0,1.861986e+10
42,17781.332917,0.004787,237.973349,0.000000,89.205319,77.720481,89.205319,0.688246,0.000659,0.004143,0.000140,0.004145,87.697932,0.000659,1.862022e+10,0.004766,3.454929e+07,1.074412,2.004067e-16,NaN,0.000000,1.466126e-16,0.0,1.0,1.0,1.862022e+10
47,17778.519859,0.004787,237.973349,0.000000,89.205319,77.720481,89.205319,0.688246,0.000659,0.004143,0.000140,0.004145,87.697930,0.000659,1.862032e+10,0.004766,3.454681e+07,1.074425,2.003720e-16,NaN,0.000000,1.465859e-16,0.0,2.0,1.0,1.862032e+10


In [36]:
hardware = Hardware(config_dict)

Optimization terminated successfully.
         Current function value: -0.689350
         Iterations: 42
         Function evaluations: 95
Trap width: (-0.04515688705444337,0.04515688705444337)
Maximum Field: 0.6893498199715139
Time to initialize field_strength_interp =  25.953555957000006 



In [ ]:
# verify slopes match times: 

band_0_df["segment_length"]* band_0_df["slope"] == band_0_df["freq"]

In [39]:
energy_loss = band_0_df["segment_length"]*band_0_df["segment_power"]*2
print(energy_loss)
delta_f_0 = sc.energy_to_freq(energy_loss,hardware.main_field)

2     4.524926e-18
7     5.755596e-19
12    4.343852e-19
17    6.397313e-19
22    5.243918e-18
27    8.081090e-18
32    3.151645e-18
37    3.231435e-19
42    3.026794e-18
47    3.699710e-18
52    1.472279e-18
57    3.211171e-18
62    8.819879e-19
67    2.075900e-18
72    2.306095e-19
77    1.356229e-18
dtype: float64


In [38]:
delta_f_0

2     1.928683e+10
7     1.928683e+10
12    1.928683e+10
17    1.928683e+10
22    1.928683e+10
27    1.928683e+10
32    1.928683e+10
37    1.928683e+10
42    1.928683e+10
47    1.928683e+10
52    1.928683e+10
57    1.928683e+10
62    1.928683e+10
67    1.928683e+10
72    1.928683e+10
77    1.928683e+10
dtype: float64

# Visualizing the tracks: 

In [11]:
trackbuilder_df = simulation.read_saved_df("TrackBuilder_tracks_df") 

In [12]:
# print(trackbuilder_df["time_stop"]-trackbuilder_df["time_start"])
# print(trackbuilder_df["segment_length"])

In [13]:
power_max = trackbuilder_df["band_power"].max()
power_min = trackbuilder_df["band_power"].min()


def power_to_color(power): 
    color = (power-power_min)/(power_max-power_min)
    return 1-color

In [15]:
%matplotlib qt


from matplotlib import colors
from matplotlib.ticker import PercentFormatter
run_length = config_dict["TrackBuilder"]["run_length"]
time_interval = 1e-8
power_max = trackbuilder_df["band_power"].max()
power_min = trackbuilder_df["band_power"].min()
print(power_max)
for index,row in trackbuilder_df.iterrows():
#     if index < 26: 
    time_start = row["time_start"]
    time_stop = row["time_stop"]
    freq_start = row["freq_start"]
    freq_stop = row["freq_stop"]
    slope = (freq_stop -freq_start)/(time_stop - time_start)
    power = row["band_power"]
    
    time =  np.arange(time_start,time_stop, time_interval)
    freq = slope*(time-time_start) + freq_start
    color = power_to_color(power)
#     color = 0
    plt.plot(time_start,freq_start,'go')
    plt.plot(time,freq, color = str(color) )



plt.figure(1)
# plt.tick_params(axis='both', which='major', labelsize=10)
plt.ylabel('Freq (Hz)', fontsize = 10)
plt.xlabel('Time (s)', fontsize = 10)
plt.title("Simulation Track Visualization")


# plt.legend()
plt.show()

1.1288456279507316e-16


In [28]:
plt.figure(1)
plt.hist(trackbuilder_df["segment_length"])
plt.yscale('log')
plt.ylabel('Freq (Hz)', fontsize = 10)
plt.xlabel('Time (s)', fontsize = 10)
plt.title("Track Visualization")


# plt.legend()
plt.show()